In [1]:
!pip3 install ipython-cypher neo4j py2neo ipycytoscape >/dev/null

In [2]:
%env NEO4J_VERSION neo4j-community-4.1.1
%env NEO4J_PATH    neo4j-community-4.1.1

env: NEO4J_VERSION=neo4j-community-4.1.1
env: NEO4J_PATH=neo4j-community-4.1.1


In [3]:
!${NEO4J_PATH}/bin/neo4j status

Neo4j is not running


In [4]:
!curl http://localhost:7474/

{
  "bolt_routing" : "neo4j://localhost:7687",
  "transaction" : "http://localhost:7474/db/{databaseName}/tx",
  "bolt_direct" : "bolt://localhost:7687",
  "neo4j_version" : "4.1.1",
  "neo4j_edition" : "community"
}

In [33]:
# criando um adapter e verificando a conexão
from py2neo import Graph

graph = Graph("bolt://127.0.0.1:7687")

graph.delete_all()
print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.relationships)))


Number of nodes: 0 
Number of relationships: 0 


In [34]:
command = """
CALL apoc.load.json('dataset/json_orders_relations.json') yield value
UNWIND value.OrderItems as items
MERGE (pedido:Pedido {id : value.order_id})
    ON CREATE SET pedido.nome = value.order_id,
                  pedido.status = value.order_status,
                  pedido.order_purchase_timestamp = value.order_purchase_timestamp,
                  pedido.order_approved_at = value.order_approved_at,
                  pedido.order_delivered_carrier_date = value.order_delivered_carrier_date,
                  pedido.order_delivered_customer_date = value.order_delivered_customer_date,
                  pedido.order_estimated_delivery_date = value.order_estimated_delivery_date
MERGE (cliente:Cliente {id : value.customer_id})
    ON CREATE SET cliente.nome = value.customer_id
MERGE (regiao : Regiao {id : value.Customer.customer_zip_code_prefix})
    ON CREATE SET regiao.nome = value.Customer.customer_zip_code_prefix
MERGE (cidade : Cidade {id : value.Customer.customer_city})
    ON CREATE SET cidade.nome = value.Customer.customer_city
MERGE (estado : Estado {id : value.Customer.customer_state})
    ON CREATE SET estado.nome = value.Customer.customer_state
MERGE (item_pedido: ItemPedido {id: value.order_id+'-'+items.order_item_id})
    ON CREATE SET item_pedido.nome = value.order_id+'-'+items.order_item_id,
                  item_pedido.preco = items.price,
                  item_pedido.shipping_limit_date = items.shipping_limit_date,
                  item_pedido.frete = items.freight_value
MERGE (produto: Produto {id : items.product_id})
    ON CREATE SET produto.nome = items.product_id,
                  produto.product_name_lenght = items.product.product_name_lenght,
                  produto.product_description_lenght = items.product.product_description_lenght,
                  produto.product_photos_qty = items.product.product_photos_qty,
                  produto.product_weight_g = items.product.product_weight_g,
                  produto.product_length_cm = items.product.product_length_cm,
                  produto.product_height_cm = items.product.product_height_cm,
                  produto.product_width_cm = items.product.product_width_cm
MERGE (fornecedor : Fornecedor {id : items.seller_id})
    ON CREATE SET fornecedor.nome = items.seller_id
MERGE (categoria : Categoria {id : items.product.product_category_name})
    ON CREATE SET categoria.nome = items.product.product_category_name
MERGE (regiaoFornecedor : Regiao {id : COALESCE(items.seller.seller_zip_code_prefix, "UNKNOWN")})
    ON CREATE SET regiaoFornecedor.nome = COALESCE(items.seller.seller_zip_code_prefix, "UNKNOWN")
MERGE (cidadeFornecedor : Cidade {id : COALESCE(items.seller.seller_city, "UNKNOWN")})
    ON CREATE SET cidadeFornecedor.nome = COALESCE(items.seller.seller_city, "UNKNOWN")
MERGE (cliente)-[:COMPROU {id_pedido : value.order_id}]->(produto)
MERGE (cliente)-[:MORA]->(regiao)
MERGE (regiao)-[:EH_PARTE_DE]->(cidade)
MERGE (cidade)-[:ESTA_LOCALIZADA]->(estado)
MERGE (produto)-[:EH_DE]->(item_pedido)
MERGE (produto)-[:EH_PRODUZIDO]->(fornecedor)
MERGE (produto)-[:EH_MONTADO]->(regiaoFornecedor)
MERGE (fornecedor)-[:PERTENCE]->(regiaoFornecedor)
MERGE (regiaoFornecedor)-[:EH_PARTE_DE]->(cidadeFornecedor)
MERGE (produto)-[:FOI_COMPRADO_POR]->(cliente)
MERGE (produto)-[:POSSUI]->(categoria)
MERGE (fornecedor)-[:FORNECE_CATEGORIA]->(categoria)
"""

result = graph.run(command)

In [46]:
# produtos mais vendidos de uma cidade
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
WITH pro.id AS product, COUNT(cliAux) as qtdeProduto
RETURN product, qtdeProduto
ORDER BY qtdeProduto DESC
"""
result = graph.run(command)

print(result.to_table())


 product                          | qtdeProduto 
----------------------------------|-------------
 87285b34884572647811a353c7ac498a |           1 
 f7e0fa615b386bc9a8b9eb52bc1fff76 |           1 
 10adb53d8faa890ca7c2f0cbcb68d777 |           1 
 72d3bf1d3a790f8874096fcf860e3eff |           1 
 5ac9d9e379c606e36a8094a6046f75dc |           1 
 2b4609f8948be18874494203496bc318 |           1 
 6893767814d1ac82a81bcd365e1cc918 |           1 
 cafd558df4c3c9d1c338ba6930ea9a62 |           1 
 6b64362e89896be7589621df54be089e |           1 
 9a469eaf45dfbc43d39ba1977a3c07af |           1 
 f35927953ed82e19d06ad3aac2f06353 |           1 
 278b3c6462e86b4556b99989513ddf73 |           1 
 30469bb5ea377eae7121981e2f0778e4 |           1 
 15a9e834e89eab39d973492882c658d6 |           1 
 a659cb33082b851fb87a33af8f0fff29 |           1 
 163e6400e6dadd0fe04775c5e9331fda |           1 
 c3ba4e8d3cb30049213b682e751e9d00 |           1 
 79da264732f717f10ebf5d102aa6c32a |           1 
 20a8603c265d777e25d

In [49]:
# categorias mais vendidas em uma cidade
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proAux)-[:POSSUI]->(cat)
WHERE proAux.id = pro.id
WITH cat.id AS categoria, COUNT(pro) as qtdeProduto
RETURN categoria, qtdeProduto
ORDER BY qtdeProduto DESC
"""

result = graph.run(command)


print(result.to_table())


 categoria              | qtdeProduto 
------------------------|-------------
 cama_mesa_banho        |           3 
 beleza_saude           |           2 
 bebes                  |           2 
 moveis_decoracao       |           2 
 telefonia              |           2 
 utilidades_domesticas  |           1 
 informatica_acessorios |           1 
 brinquedos             |           1 
 eletronicos            |           1 
 livros_interesse_geral |           1 
 eletroportateis        |           1 
 esporte_lazer          |           1 
 automotivo             |           1 
 relogios_presentes     |           1 



In [51]:
# cidades que venderam mais produtos
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[r:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
WITH cid.id as city, COUNT(pro.id) as qtdeProduto
RETURN city, qtdeProduto
ORDER BY qtdeProduto DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                  | qtdeProduto 
-----------------------|-------------
 sao paulo             |          20 
 rio de janeiro        |           7 
 sorocaba              |           3 
 brasilia              |           3 
 salvador              |           2 
 santo andre           |           2 
 juiz de fora          |           2 
 jundiai               |           2 
 ribeirao preto        |           2 
 carapicuiba           |           2 
 santa rosa            |           1 
 goiania               |           1 
 imbituba              |           1 
 hortolandia           |           1 
 ouro preto            |           1 
 congonhinhas          |           1 
 vianopolis            |           1 
 mesquita              |           1 
 palmares              |           1 
 boa vista             |           1 
 sao carlos            |           1 
 santa barbara d'oeste |           1 
 alvorada              |           1 
 ibiapina              |           1 
 faxinalzinh

In [56]:
# Cidades que apresentaram o maior volume de vendas
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = pro.id
WITH cidade.id as city, SUM(item.preco) as valorVendas
RETURN city, ROUND(valorVendas*100)/100 as vendas
ORDER BY valorVendas DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                    |  vendas 
-------------------------|---------
 sao paulo               | 2463.81 
 rio de janeiro          |  2367.4 
 boninal                 |   659.8 
 tapiratiba              |   544.0 
 jaboatao dos guararapes |   445.0 
 ribeirao preto          |  419.89 
 duque de caxias         |   369.0 
 presidente venceslau    |   339.0 
 juiz de fora            |   318.0 
 sorocaba                |  289.79 
 barreiras               |   238.6 
 itajai                  |   219.9 
 portao                  |   219.0 
 santo andre             |  198.89 
 cascavel                |   190.0 
 paulinia                |   190.0 
 palmares                |   186.9 
 antonio cardoso         |  179.99 
 santa barbara d'oeste   |   179.0 
 vianopolis              |   159.9 
 brasilia                |   154.8 
 congonhinhas            |   147.9 
 goiania                 |   132.4 
 salvador                |   130.4 
 blumenau                |  129.99 



In [58]:
# Cidades que apresentaram o maior valor medio de frete cobrado em cada venda
command = """
MATCH (cidade:Cidade)
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = pro.id
WITH cidade.id as city, AVG(item.frete) as valorFrete
RETURN city, ROUND(valorFrete*100)/100 as Frete
ORDER BY valorFrete DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())


 city                    | Frete 
-------------------------|-------
 boninal                 | 82.48 
 jaboatao dos guararapes | 63.17 
 blumenau                | 57.58 
 antonio cardoso         | 51.13 
 salvador                | 48.64 
 palmares                |  38.0 
 alto santo              | 37.37 
 paracambi               |  34.2 
 boa vista               | 33.24 
 tapiratiba              | 30.36 
 alvorada                | 28.95 
 congonhinhas            | 27.36 
 sao goncalo do amarante |  27.2 
 ibiapina                | 26.89 
 rio de janeiro          | 26.28 
 santa luzia             | 24.84 
 aracaju                 | 22.85 
 luis eduardo magalhaes  | 22.62 
 presidente kubitschek   |  22.4 
 gravatai                | 21.27 
 paulinia                | 19.41 
 vianopolis              | 19.22 
 itajai                  | 18.79 
 flor da serra do sul    | 18.57 
 paranavai               | 18.47 



In [59]:
# Cidades que apresentaram o maior valor medio de frete cobrado em cada venda
command = """
MATCH (cidade:Cidade)
WHERE cidade.id = "sao paulo"
MATCH (reg)-[:EH_PARTE_DE]->(cid)
WHERE cid.id = cidade.id
MATCH (cli)-[:MORA]->(regAux)
WHERE regAux.id = reg.id
MATCH (pro)-[:FOI_COMPRADO_POR]->(cliAux)
WHERE cliAux.id = cli.id
MATCH (proAux)-[:POSSUI]->(cat)
WHERE proAux.id = pro.id
MATCH (proItem)-[:EH_DE]->(item)
WHERE proItem.id = proAux.id
WITH cidade.id as city, cat.id AS categoria, AVG(item.frete) as valorFrete
RETURN city, categoria, ROUND(valorFrete*100)/100 as Frete
ORDER BY valorFrete DESC
LIMIT 25
"""

result = graph.run(command)

print(result.to_table())

 city      | categoria              | Frete 
-----------|------------------------|-------
 sao paulo | esporte_lazer          | 28.15 
 sao paulo | bebes                  | 21.77 
 sao paulo | informatica_acessorios | 16.13 
 sao paulo | brinquedos             | 16.11 
 sao paulo | livros_interesse_geral | 15.56 
 sao paulo | eletroportateis        | 13.47 
 sao paulo | moveis_decoracao       | 13.45 
 sao paulo | automotivo             | 13.25 
 sao paulo | cama_mesa_banho        | 12.45 
 sao paulo | telefonia              | 10.24 
 sao paulo | beleza_saude           |  8.94 
 sao paulo | utilidades_domesticas  |  8.72 
 sao paulo | relogios_presentes     |  7.91 
 sao paulo | eletronicos            |  7.78 

